<a href="https://colab.research.google.com/github/31indianaSKKU/NBA_CareerLength/blob/main/NCAA_Stat_by_Season%26School%26Player_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 2000년대에서만 발생하는 문제
-> 2000년대와 최근 테이블의 컬럼이 달라서 예외처리가 제대로 동작하지 않았습니다.
-> -> merge하기 전에 컬럼을 드랍하여 번거로운 rename과정을 제거
-> -> 한번에 drop하면 없는 컬럼에서 오류가 발생하기에, 하나씩 돌면서 에러가 발생하더라도 넘어가도록 처리

* Season컴럼의 표시형식 문제는 재현되지 않아 확인 하지 못햇습니다.
-> 혹시 엑셀로 csv를 보고 계시다면 엑셀 이슈일 수 있습니다.
-> 데이터에는 이상 없어 보입니다.


In [ ]:
#수정 후 코드
# 쥬피터 OK. 
# But colab Not OK. Why?
#NCAA데이터 크롤링 완성



import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import time
import os

def get_season_from_page(soup):
    season_element = soup.select_one("#meta div h1 span")
    if season_element:
        return season_element.text.strip().split()[0]
    return ""

def get_school_name(url):
    url_parts = url.split("/")
    return url_parts[-3]


def get_table_data(table):
    header_row = [cell.text.strip() for cell in table.select("thead tr th")]
    data_rows = table.select("tbody tr")
    data = [[cell.text.strip() for cell in row.select("th, td")] for row in data_rows]
    return pd.DataFrame(data, columns=header_row)


for yrs in range(2000,2001):
    ratings_url = f"https://www.sports-reference.com/cbb/seasons/men/{yrs}-ratings.html" 
    response = requests.get(ratings_url)
    soup = BeautifulSoup(response.content, "html.parser")
    base_url = "https://www.sports-reference.com/"

    response = requests.get(ratings_url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table", id="ratings")
#     rows = table.select("tbody tr")
    # rows = table.select("tbody tr[data-row]")[:5]  # 첫 5개 행만 선택
    rows = table.select("tbody tr")[:2]  # 첫 5개 행만 선택

    # 빈 데이터프레임 생성
    combined_df = pd.DataFrame()
   # 필요한 컬럼들을 리스트로 저장
    columns_to_drop = ["#", "Class", "Pos", "Height", "Weight", "Hometown", "High School", "RSCI Top 100", "Summary"]

    for row in rows:
        link_element = row.select_one("td:nth-of-type(1) a")
        if link_element:
            link = link_element["href"]
            full_url = base_url + link

            response = requests.get(full_url)
            soup = BeautifulSoup(response.content, "html.parser")

            roster_table = soup.find("table", id="roster")
            per_game_table = soup.find("table", id="per_game")
            totals_table = soup.find("table", id="totals")
            per_min_table = soup.find("table", id="per_min")
            per_poss_table = soup.find("table", id="per_poss")
            advanced_table = soup.find("table", id="advanced")

            data_frames = list()

            if roster_table is not None:
                roster_data = get_table_data(roster_table)
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                roster_data["Season"] = season
                roster_data["School"] = school
                data_frames.append(roster_data)
            else:
                print(f"{season} {school} Roster table not found.")
                pass

            if per_game_table is not None:
                per_game_data = get_table_data(per_game_table)
                per_game_data = per_game_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_per_game")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                per_game_data["Season"] = season
                per_game_data["School"] = school
                data_frames.append(per_game_data)
            else:
                print(f"{season} {school} PerGame table not found.")
                pass

            if totals_table is not None:
                totals_data = get_table_data(totals_table)
                totals_data = totals_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_totals")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                totals_data["Season"] = season
                totals_data["School"] = school
                # print("Totals data frame:")
                # print(totals_data.head())
                data_frames.append(totals_data)
            else:
                print(f"{season} {school} Totals table not found.")
                pass

            if per_min_table is not None:
                per_min_data = get_table_data(per_min_table)
                per_min_data = per_min_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_per_min")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                per_min_data["Season"] = season
                per_min_data["School"] = school
                # print("Per 40 Min data frame:")
                # print(per_min_data.head())
                data_frames.append(per_min_data)
            else:
                print(f"{season} {school} Per40Min table not found.")
                pass

            if per_poss_table is not None:
                per_poss_data = get_table_data(per_poss_table)
                per_poss_data = per_poss_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_per_poss")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                per_poss_data["Season"] = season
                per_poss_data["School"] = school
                # print("Per 100 Poss data frame:")
                # print(per_poss_data.head())
                data_frames.append(per_poss_data)
            else:
                print(f"{season} {school} Per100Poss table not found.")
                pass

            if advanced_table is not None:
                advanced_data = get_table_data(advanced_table)
                advanced_data = advanced_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_advanced")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                advanced_data["Season"] = season 
                advanced_data["School"] = school
                # print("Advanced data frame:")
                # print(advanced_data.head())
                data_frames.append(advanced_data)
            else:
                print(f"{season} {school} Advanced table not found.")
                pass

            combined_df = roster_data.copy()
            for col in columns_to_drop:
                try:
                    combined_df = combined_df.drop(col, axis=1)
                except:
                    continue

            for df in data_frames:
                if df is not None:
                    df = df.loc[:, ~df.columns.duplicated()]
                    combined_df = pd.merge(combined_df, df, on=["Player", "Season", "School"], how="outer")

            combined_df = combined_df[["Season", "School"] + [col for col in combined_df.columns if col not in ["Season", "School"]]]
            # Season 값 형식 변경
            # combined_df["Season"] = combined_df["Season"].apply(lambda x: f"{x[:4]}-{x[-2:]}")
            combined_df["Season"] = combined_df["Season"].astype(str)

            # 파일명 생성
            filename = "NCAA_" + season + ".csv"
            #  # 빈 데이터프레임을 CSV 파일로 저장
            # df.to_csv(filename, index=False)

            # 파일이 이미 존재하는 경우
            if filename in os.listdir():
                # 기존 파일 읽어오기
                existing_df = pd.read_csv(filename)
                # 기존 데이터와 현재 데이터 병합
                combined_df = pd.concat([existing_df, combined_df], ignore_index=False)
            # else:
                # combined_df = pd.concat([combined_df] + data_frames, ignore_index=False)

            # CSV 파일로 저장
            combined_df.to_csv(filename, index=False)
            print("Combined DataFrame has been saved as", filename)

            # 일정 시간 대기
            time.sleep(5)  # 5초 대기

print("Crawling complete!")

OptionError: ignored

질문2.
1. 누구나 접속 가능한 웹페이지(ex. https://www.sports-reference.com/cbb/seasons/men/2022-ratings.html)

2. 목적: 첫번째 페이지 내의 테이블(시즌별 학교순위)에서 school에 링크로 연결된 페이지를 방문하여 roster, total, per game 등의 테이블 데이터를 수집. 이 과정을 학교별 시즌별로 반복 수행. 

3. 문제상황: for yrs in range(season_from,season_to): 2019년처럼 최근의 시즌들에 대해서는 원하는 데이터를 가져오고 있으나 2000대 초반을 돌려보면 문제현상 발생.

문제현상1. 컬럼명 Season의 표시형식이 연도가 아닌 월-일로 저장.

문제현상2. 각각의 테이블을 데이터프레임으로 합치는 과정에서 combined_df = roster_data.copy()코드로 인해 roster_data가 중복으로 생성되는데 이에 대한 처리가 안됨: 컬럼명_x와 컬럼명_y가 생성되어 컬럼명_y들의 삭제처리와 컬럼명_x들의 "_x"삭제처리가 안됨.

In [ ]:
#수정 전 코드
#아래의 코드 문제점
#1. 2000년대 시즌 error: combined_df = combined_df.drop(["#_y", "Class_y", "Pos_y", "Height_y", "Weight_y", "Hometown_y", "High School_y", "RSCI Top 100_y", "Summary_y"], axis=1)
#2. Season컬럼이 표시 형식이 2009-10시즌의 경우 Oct-09 식으로 표시됨.
#3. 현재 1의 error는 drop할 컬럼명을 먼저 리스트로 만들어 놓고 컬럼명에 해당 컬럼이 있을 경우 삭제 처리하려고 시도 중이나 반영안되고 있음.


import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import time
import os

def get_season_from_page(soup):
    season_element = soup.select_one("#meta div h1 span")
    if season_element:
        return season_element.text.strip().split()[0]
    return ""

def get_school_name(url):
    url_parts = url.split("/")
    return url_parts[-3]


def get_table_data(table):
    header_row = [cell.text.strip() for cell in table.select("thead tr th")]
    data_rows = table.select("tbody tr")
    data = [[cell.text.strip() for cell in row.select("th, td")] for row in data_rows]
    return pd.DataFrame(data, columns=header_row)


for yrs in range(2005,2007):
    ratings_url = f"https://www.sports-reference.com/cbb/seasons/men/{yrs}-ratings.html" 
    response = requests.get(ratings_url)
    soup = BeautifulSoup(response.content, "html.parser")
    base_url = "https://www.sports-reference.com/"

    response = requests.get(ratings_url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table", id="ratings")
    # rows = table.select("tbody tr")
    # rows = table.select("tbody tr[data-row]")[:5]  # 첫 5개 행만 선택
    rows = table.select("tbody tr")[:2]  # 첫 5개 행만 선택

    # 빈 데이터프레임 생성
    combined_df = pd.DataFrame()
   # 필요한 컬럼들을 리스트로 저장
    columns_to_drop = ["#_y", "Class_y", "Pos_y", "Height_y", "Weight_y", "Hometown_y", "High School_y", "RSCI Top 100_y", "Summary_y"]

    for row in rows:
        link_element = row.select_one("td:nth-of-type(1) a")
        if link_element:
            link = link_element["href"]
            full_url = base_url + link

            response = requests.get(full_url)
            soup = BeautifulSoup(response.content, "html.parser")

            roster_table = soup.find("table", id="roster")
            per_game_table = soup.find("table", id="per_game")
            totals_table = soup.find("table", id="totals")
            per_min_table = soup.find("table", id="per_min")
            per_poss_table = soup.find("table", id="per_poss")
            advanced_table = soup.find("table", id="advanced")

            if roster_table is not None:
                roster_data = get_table_data(roster_table)
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                roster_data["Season"] = season
                roster_data["School"] = school
            else:
                print(f"{season} Roster table not found.")
                pass

            if per_game_table is not None:
                per_game_data = get_table_data(per_game_table)
                per_game_data = per_game_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_per_game")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                per_game_data["Season"] = season
                per_game_data["School"] = school
            else:
                print(f"{season} Per Game table not found.")
                pass

            if totals_table is not None:
                totals_data = get_table_data(totals_table)
                totals_data = totals_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_totals")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                totals_data["Season"] = season
                totals_data["School"] = school
                # print("Totals data frame:")
                # print(totals_data.head())
            else:
                print(f"{season} Totals table not found.")
                pass

            if per_min_table is not None:
                per_min_data = get_table_data(per_min_table)
                per_min_data = per_min_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_per_min")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                per_min_data["Season"] = season
                per_min_data["School"] = school
                # print("Per 40 Min data frame:")
                # print(per_min_data.head())
            else:
                print(f"{season} Per 40 Min table not found.")
                pass

            if per_poss_table is not None:
                per_poss_data = get_table_data(per_poss_table)
                per_poss_data = per_poss_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_per_poss")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                per_poss_data["Season"] = season
                per_poss_data["School"] = school
                # print("Per 100 Poss data frame:")
                # print(per_poss_data.head())
            else:
                print(f"{season} Per 100 Poss table not found.")
                pass

            if advanced_table is not None:
                advanced_data = get_table_data(advanced_table)
                advanced_data = advanced_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_advanced")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                advanced_data["Season"] = season 
                advanced_data["School"] = school
                # print("Advanced data frame:")
                # print(advanced_data.head())
            else:
                print(f"{season} Advanced table not found.")
                pass

            # data_frames = [roster_data, per_game_data]
            # combined_df = pd.concat([combined_df] + data_frames, ignore_index=True, sort=False)


            # Merge all data frames
            data_frames = [roster_data, per_game_data, totals_data, per_min_data, per_poss_data, advanced_data]
############################################################################################################################################################################### 
###############################################################################################################################################################################
###############################################################################################################################################################################
###############################################################################################################################################################################  
#문제 부분 
            combined_df = roster_data.copy()  


            for df in data_frames:
                if df is not None:
                    df = df.loc[:, ~df.columns.duplicated()]
                    combined_df = pd.merge(combined_df, df, on=["Player", "Season", "School"], how="outer")

            # combined_df = combined_df.drop(["#_y", "Class_y", "Pos_y", "Height_y", "Weight_y", "Hometown_y", "High School_y", "RSCI Top 100_y", "Summary_y"], axis=1)
            # combined_df.columns = combined_df.columns.str.replace("_x", "")
            # 컬럼들이 존재하는 경우에만 해당 컬럼 제거 및 이름 수정
                    if all(col in combined_df.columns for col in columns_to_drop):
                       combined_df = combined_df.drop(columns_to_drop, axis=1)
                       combined_df.columns = combined_df.columns.str.replace("_x", "")

            combined_df = combined_df[["Season", "School"] + [col for col in combined_df.columns if col not in ["Season", "School"]]]
            # Season 값 형식 변경
            # combined_df["Season"] = combined_df["Season"].apply(lambda x: f"{x[:4]}-{x[-2:]}")
            combined_df["Season"] = combined_df["Season"].astype(str)
############################################################################################################################################################################### 
###############################################################################################################################################################################
###############################################################################################################################################################################
###############################################################################################################################################################################  

            # # 필요한 컬럼들을 리스트로 저장
            # columns_to_drop = ["#_y", "Class_y", "Pos_y", "Height_y", "Weight_y", "Hometown_y", "High School_y", "RSCI Top 100_y", "Summary_y"]


            # 파일명 생성
            filename = "NCAA_" + season + ".csv"
            #  # 빈 데이터프레임을 CSV 파일로 저장
            # df.to_csv(filename, index=False)

            # 파일이 이미 존재하는 경우
            if filename in os.listdir():
                # 기존 파일 읽어오기
                existing_df = pd.read_csv(filename)
                # 기존 데이터와 현재 데이터 병합
                combined_df = pd.concat([existing_df, combined_df], ignore_index=False)
            # else:
                # combined_df = pd.concat([combined_df] + data_frames, ignore_index=False)

            # CSV 파일로 저장
            combined_df.to_csv(filename, index=False)
            print("Combined DataFrame has been saved as", filename)

            # 일정 시간 대기
            time.sleep(5)  # 5초 대기

print("Crawling complete!")